### Подготовка

Установим библиотеку pygmo, импортируем её и другие нужные библиотеки

In [179]:
!pip install -q pygmo
import pygmo as pg

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

Выберем следующие алгоритмы одномерной безусловной оптимизации:
* Самоадаптивная дифференциальная эволюция (Self-adaptive differentional evolution)
* Алгоритм серой волчьей стаи (Gray wolf optimizer)
* Алгоритм пчелиной колонии (Artificial bee colony)
* Алгоритм оптимизации роя частиц (Particle swarm optimization)
* Адаптивно-эволюционная стратегия ковариационной матрицы (Covariation matrix evolutionary strategy)

Каждый алгоритм будет запускаться 1000 раз и при каждом запуске генерировать 1000 точек


In [186]:
num_gen = 1000
xtol, ftol = [1e-300 for _ in range(2)]

algorithms = [
    pg.algorithm(pg.sade(gen=num_gen, xtol=xtol, ftol=ftol)),
    pg.algorithm(pg.gwo(gen=num_gen)),
    pg.algorithm(pg.bee_colony(gen=num_gen, limit=num_gen)),
    pg.algorithm(pg.pso(gen=num_gen)),
    pg.algorithm(pg.cmaes(gen=num_gen, xtol=xtol, ftol=ftol))
]

names = [algorithm.get_name() for algorithm in algorithms]
names = [name[(name.find(':') + 2):] for name in names]

Объявим функции Матьяса и Гольдстейна-Прайса, их точки оптимума и границы начальных точек

In [187]:
def mattias(x, y):
    return 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y


mattias_opt = np.zeros(2)
mattias_bounds = [[-10, 10], [-10, 10]]


def goldstein_price(x, y):
    return (1 + np.square(x + y + 1) * (19 - 14 * x + 3 * np.square(x) - 14 * y + 6 * x * y + 3 * np.square(y))) * \
           (30 + np.square(x * 2 - y * 3) * (18 - 32 * x + 12 * np.square(x) + 48 * y - 36 * x * y + 27 * np.square(y)))


goldstein_price_opt = np.array([0, -1])
goldstein_price_bounds = [[-2, 2], [-2, 2]]

Объявим функцию, которая будет подсчитывать результат метрики RMSE между предсказаниями и истинными точками оптимума функции

In [188]:
def RMSE(coords, opt):
    return np.sqrt(np.mean(np.square(coords - opt), axis=1))

Объявим класс Problem, объекты которого будут подаваться в качетве аргумента в функцию pg.problem

In [189]:
class Problem():
     def __init__(self, loss, dim, bounds, name=None):
         self.loss = loss
         self.dim = dim
         self.bounds = bounds
         self.name = name

     def fitness(self, input):
         return [self.loss(*input)]

     def get_bounds(self):
         return tuple(map(list, zip(*self.bounds)))

     def get_name(self):
        return self.name or 'Unknown'

     def get_extra_info(self):
         return '\tDimensions: ' + str(self.dim)

Объявим функцию, которая будет для переданной функции для каждого алгоритма вычислять точку оптимума, значение функции в ней и значение метрики RMSE между предсказанным и истинным значением оптимума

In [190]:
def get_results(loss, dim, bounds, name, opt, size=num_gen):
    results = np.empty(shape=[0, dim + 1])
    problem = pg.problem(Problem(loss=loss, dim=dim, bounds=bounds, name=name))

    pbar = tqdm(zip(algorithms, names))
    for algorithm, name in pbar:
        pbar.set_description(f'Processing {name}')
        pop = algorithm.evolve(pg.population(problem, size=size))
        local_results = np.concatenate((pop.champion_x, pop.champion_f))
        results = np.vstack((results, local_results[np.newaxis, :]))

    metrics = RMSE(results[:, :dim], opt)
    results = np.hstack((results, metrics[:, np.newaxis]))
    return pd.DataFrame(results, columns=['x', 'y', 'f(x, y)', 'RMSE'], index=names)

### Непосредственное обучение

In [191]:
mattias_results = get_results(
    loss=mattias, dim=2, bounds=mattias_bounds, name='Функция Матьяса',
    opt=mattias_opt
)
mattias_results

0it [00:00, ?it/s]

,x,y,"f(x, y)",RMSE
Self-adaptive Differential Evolution,2.963674e-76,3.176396e-76,3.883172e-153,3.071877e-76
Grey Wolf Optimizer,1.819136e-162,-6.038180e-163,0.000000e+00,0.000000e+00
Artificial Bee Colony,-4.916289e-07,7.249872e-08,8.131669e-14,3.513937e-07
Particle Swarm Optimization,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Covariance Matrix Adaptation Evolutionary Strategy,1.275427e-23,1.771796e-23,1.544512e-47,1.543693e-23


In [192]:
goldstein_price_results = get_results(
    loss=goldstein_price, dim=2, bounds=goldstein_price_bounds,
    name='Функция Гольдстейна-Прайса', opt=goldstein_price_opt
)
goldstein_price_results

0it [00:00, ?it/s]

,x,y,"f(x, y)",RMSE
Self-adaptive Differential Evolution,-4.908612e-09,-1.000000,3.000000,4.513432e-09
Grey Wolf Optimizer,-2.998119e-06,-1.000001,3.000000,2.165494e-06
Artificial Bee Colony,3.947832e-09,-1.000000,3.000000,4.743398e-09
Particle Swarm Optimization,2.053579e-09,-1.000000,3.000000,1.928226e-09
Covariance Matrix Adaptation Evolutionary Strategy,-9.490069e-05,-1.000143,3.000008,1.215842e-04


### Выводы

В данной лабораторной работе я использовал на практике 5 эволюционных методов для решения задачи одномерной безусловной оптимизации

Лучше всех показал себя алгоритм оптимизации роя частиц - для функции Матьяса он вообще попал в сам оптимум